In [ ]:
# python -m venv venv
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [1]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [24]:
# If modifying these scopes, delete the file token.json.
# SCOPES = [    
#     'https://www.googleapis.com/auth/classroom.announcements',
#     'https://www.googleapis.com/auth/classroom.announcements.readonly',
#     'https://www.googleapis.com/auth/classroom.courses',
#     'https://www.googleapis.com/auth/classroom.courses.readonly',
#     'https://www.googleapis.com/auth/classroom.coursework.me',
#     'https://www.googleapis.com/auth/classroom.coursework.me.readonly',
#     'https://www.googleapis.com/auth/classroom.coursework.students',
#     'https://www.googleapis.com/auth/classroom.coursework.students.readonly',
#     'https://www.googleapis.com/auth/classroom.courseworkmaterials',
#     'https://www.googleapis.com/auth/classroom.courseworkmaterials.readonly',
#     'https://www.googleapis.com/auth/classroom.guardianlinks.me.readonly',
#     'https://www.googleapis.com/auth/classroom.guardianlinks.students',
#     'https://www.googleapis.com/auth/classroom.guardianlinks.students.readonly',
#     'https://www.googleapis.com/auth/classroom.profile.emails',
#     'https://www.googleapis.com/auth/classroom.profile.photos',
#     'https://www.googleapis.com/auth/classroom.push-notifications',
#     'https://www.googleapis.com/auth/classroom.rosters',
#     'https://www.googleapis.com/auth/classroom.rosters.readonly',
#     'https://www.googleapis.com/auth/classroom.student-submissions.me.readonly',
#     'https://www.googleapis.com/auth/classroom.student-submissions.students.readonly',
#     'https://www.googleapis.com/auth/classroom.topics',
#     'https://www.googleapis.com/auth/classroom.topics.readonly'
# ]

SCOPES = ['https://www.googleapis.com/auth/classroom.courses.readonly']

In [25]:
def read():
    """Shows basic usage of the Classroom API.
    Prints the names of the first 10 courses the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("classroom", "v1", credentials=creds)

        # Call the Classroom API
        results = service.courses().list(pageSize=10).execute()
        courses = results.get("courses", [])

        if not courses:
            print("No courses found.")
            return
        # Prints the names of the first 10 courses.
        print("Courses:")
        for course in courses:
            print(course["name"])

    except HttpError as error:
        print(f"An error occurred: {error}")

In [26]:
read()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=236633569803-imjj2d5t7jemnnqdhkea2i6k4mdfhtl1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56034%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.courses.readonly&state=GF2rVBNKKP2Dmwt6U2icClfqBs4A36&access_type=offline


TimeoutError: [WinError 10060] Uma tentativa de conexão falhou porque o componente conectado não respondeu
corretamente após um período de tempo ou a conexão estabelecida falhou
porque o host conectado não respondeu

In [22]:
def list_courses():
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    try:
        service = build("classroom", "v1", credentials=creds)
        results = service.courses().list().execute()
        courses = results.get('courses', [])

        if not courses:
            print('No courses found.')
        else:
            print('Courses:')
            for course in courses:
                print(f'{course["name"]} ({course["id"]})')
    except Exception as e:
        print(f'An error occurred: {e}')

list_courses()

An error occurred: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.


In [ ]:
# pip install tenacity

In [15]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from tenacity import retry, stop_after_attempt, wait_fixed

SCOPES = [
    'https://www.googleapis.com/auth/classroom.courses.readonly'
]

@retry(stop=stop_after_attempt(3), wait=wait_fixed(2))
def read():
    """Shows basic usage of the Classroom API.
    Prints the names of the first 10 courses the user has access to.
    """
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("classroom", "v1", credentials=creds)
        results = service.courses().list(pageSize=10).execute()
        courses = results.get("courses", [])

        if not courses:
            print("No courses found.")
            return
        print("Courses:")
        for course in courses:
            print(course["name"])

    except HttpError as error:
        print(f"An error occurred: {error}")

read()

RetryError: RetryError[<Future at 0x23476a6a990 state=finished raised TimeoutError>]